In [ ]:
import os
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

FIGSIZE = (10,5)
# Lista de configurações para testar
# Ordem: taxa_chegada, tempo_medio, num_servidores, tempo_observação
CENARIOS_QUESTAO_2 = [[x, 0.88, 10, 10] for x in range(1, 15, 1)]

def rodar_simulador(taxa_chegada, tempo_medio_servico, num_servidores, tempo_observacao):
    comando = [
        "java", "-cp", "bin:lib/*", "ServidorWeb",
        str(taxa_chegada), 
        str(tempo_medio_servico), 
        str(num_servidores), 
        str(tempo_observacao)
    ]

    resultado = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Divide a saída em linhas
    linhas = resultado.stdout.splitlines()

    # Captura a primeira e a segunda linha (se existirem)
    primeira_saida = linhas[0] if len(linhas) > 0 else None
    segunda_saida = linhas[1] if len(linhas) > 1 else None

    return primeira_saida, segunda_saida, resultado.stderr


In [ ]:
dados = []
colunas = None

for cenario in CENARIOS_QUESTAO_2:
    saida1, saida2, erro = rodar_simulador(*cenario)

    if saida1 and saida2:
        if colunas is None:
            colunas = saida1.split()

        valores = saida2.split()
        print(f'Testando cenario: {cenario}')
        dados.append(valores)
df = pd.DataFrame(dados, columns=colunas)

DEFAULT_FILE_NAME = "saida_simulador.csv"
df.to_csv(f"{DEFAULT_FILE_NAME}", index=False, sep=";")

print(f"CSV gerado com sucesso: {DEFAULT_FILE_NAME}")

In [ ]:
questao_2_df = pd.read_csv(DEFAULT_FILE_NAME, sep=";", decimal=",")
questao_2_df = questao_2_df.sort_values(by="TaxaDeChegadaMedia", ascending=True)

# Usando o modelo questão 2

In [ ]:
def fatorial(n):
    if n == 0 or n == 1:
        return 1
    resultado = 1
    for i in range(2, n + 1):
        resultado *= i
    return resultado

def calculo_modelo(taxaChegada, tempoServico, numServidores):
    c = numServidores
    taxaServico = 1 / tempoServico
    u = taxaServico
    p=taxaChegada/ c * u
    sum_term = sum((taxaChegada / taxaServico) ** n / fatorial(n) for n in range(numServidores))
    P0 = 1 / (sum_term + ((taxaChegada / taxaServico) ** numServidores / (fatorial(numServidores) * (1 - p))))


    Pq = (taxaChegada/u) ** c / fatorial(c) * 1 / 1 - p * P0
    Lq = Pq * p / 1 - p
    W = (Lq / taxaChegada) + (1 / taxaServico)
    
    return W

# Testando para diferentes taxas de chegada usando o modelo
taxas_chegada = list(range(1, 15, 1))

tempo_servico = 0.88
num_servidores = 10

resultados_modelo = []
for taxa in taxas_chegada:
    W = calculo_modelo(taxa, tempo_servico, num_servidores)
    resultados_modelo.append({"Taxa de Chegada": taxa, "Tempo Médio de Resposta": W/1000})
    print(f"Taxa de chegada = {taxa}, Tempo médio de resposta W = {W/100:.2f} seg")

df_resultados = pd.DataFrame(resultados_modelo)

plt.figure(figsize=(FIGSIZE))
sns.set_theme(style="whitegrid")
plt.title('Modelo')
sns.lineplot(data=df_resultados, x="Taxa de Chegada", y="Tempo Médio de Resposta", linewidth=1)
plt.xticks(rotation=45)
plt.show()



# Usando a simulação

In [ ]:
questao_2_df = questao_2_df.sort_values(by="TempoMedioDeResposta", ascending=False)

plt.figure(figsize=(FIGSIZE))
sns.set_theme(style="whitegrid")
plt.title('Simulação')
sns.lineplot(data=questao_2_df, x="TaxaDeChegadaMedia", y="TempoMedioDeResposta", linewidth=1)
plt.xticks(rotation=45)
plt.show()


In [ ]:
questao_2_df